In [1]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4335    0  4335    0     0  11385      0 --:--:-- --:--:-- --:--:-- 11377
100  517k  100  517k    0     0   614k      0 --:--:-- --:--:-- --:--:--  614k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4347    0  4347    0     0  17816      0 --:--:-- --:--:-- --:--:-- 17888
100  392k  100  392k    0     0   645k      0 --:--:-- --:--:-- --:--:-- 1915k


In [2]:
import numpy as np
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/Nifty.csv')

In [4]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-07-15,16010.799805,16066.950195,15927.299805,16049.200195,16049.200195,205800
1,2022-07-18,16151.400391,16287.950195,16142.200195,16278.500000,16278.500000,227700
2,2022-07-19,16187.049805,16359.500000,16187.049805,16340.549805,16340.549805,222600
3,2022-07-20,16562.800781,16588.000000,16490.949219,16520.849609,16520.849609,284400
4,2022-07-21,16523.550781,16626.949219,16483.900391,16605.250000,16605.250000,243600
...,...,...,...,...,...,...,...
242,2023-07-10,19400.349609,19435.849609,19327.099609,19355.900391,19355.900391,268200
243,2023-07-11,19427.099609,19515.099609,19406.449219,19439.400391,19439.400391,251300
244,2023-07-12,19497.449219,19507.699219,19361.750000,19384.300781,19384.300781,327900
245,2023-07-13,19495.199219,19567.000000,19385.800781,19413.750000,19413.750000,310400


In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df['DoW'] = df['Date'].dt.weekday/6
df.sort_values(['Date'], inplace=True)

In [6]:
def add_features(t_df):
    # Returns in the last t intervals
    for t in range(1, 11):
        t_df[f'ret{t}'] = t_df['Close'].div(t_df['Open'].shift(t-1)).sub(1)
    # Simple Moving Average based features
        t_df['sma'] = talib.SMA(t_df['Close'])
        t_df['5sma'] = talib.SMA(t_df['Close'], timeperiod=5)
        t_df['20sma'] = talib.SMA(t_df['Close'], timeperiod=20)
    t_df['sma_buy'] = np.where((t_df['Close'] < t_df['5sma']), 1, 0)

In [7]:
updated_df = add_features(df)


In [8]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,DoW,ret1,sma,...,ret2,ret3,ret4,ret5,ret6,ret7,ret8,ret9,ret10,sma_buy
0,2022-07-15,16010.799805,16066.950195,15927.299805,16049.200195,16049.200195,205800,0.666667,0.002398,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2022-07-18,16151.400391,16287.950195,16142.200195,16278.500000,16278.500000,227700,0.000000,0.007869,NaN,...,0.016720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2022-07-19,16187.049805,16359.500000,16187.049805,16340.549805,16340.549805,222600,0.166667,0.009483,NaN,...,0.011711,0.020595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2022-07-20,16562.800781,16588.000000,16490.949219,16520.849609,16520.849609,284400,0.333333,-0.002533,NaN,...,0.020621,0.022874,0.031857,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2022-07-21,16523.550781,16626.949219,16483.900391,16605.250000,16605.250000,243600,0.500000,0.004944,NaN,...,0.002563,0.025835,0.028100,0.037128,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2023-07-10,19400.349609,19435.849609,19327.099609,19355.900391,19355.900391,268200,0.000000,-0.002291,18844.138411,...,-0.003444,-0.001537,-0.002579,-0.002612,0.005684,0.014628,0.023680,0.032394,0.036053,1
243,2023-07-11,19427.099609,19515.099609,19406.449219,19439.400391,19439.400391,251300,0.166667,0.000633,18872.163411,...,0.002013,0.000855,0.002770,0.001724,0.001690,0.010023,0.019005,0.028096,0.036848,0
244,2023-07-12,19497.449219,19507.699219,19361.750000,19384.300781,19384.300781,327900,0.333333,-0.005803,18897.178451,...,-0.002203,-0.000827,-0.001982,-0.000072,-0.001116,-0.001149,0.007160,0.016116,0.025182,1
245,2023-07-13,19495.199219,19567.000000,19385.800781,19413.750000,19413.750000,310400,0.500000,-0.004178,18926.490104,...,-0.004293,-0.000687,0.000691,-0.000466,0.001447,0.000402,0.000368,0.008690,0.017660,0
